In [1]:
import pandas as pd
import math
import datetime
from datetime import datetime
from datetime import date
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pickle


In [2]:
def ReadXlsx(CGM,Insulin):
    
    #creating the matrix with labels below
    filename1 = pd.read_excel(CGM)
    filename2 = pd.read_excel(Insulin)
    #print(filename2.Index[0])
    CGMData_pre = []
    InsulinData_pre = []

    eatingBehavior = filename1['Sensor Glucose (mg/dL)'].to_numpy()
    ISIG = filename1['ISIG Value'].to_numpy()
    CGMDate = filename1['Date'].to_numpy()
    CGMTime = filename1['Time'].to_numpy()


    Eats =  filename2['BWZ Carb Input (grams)'].to_numpy()
    InsulinDate = filename2['Date'].to_numpy()
    InsulinTime = filename2['Time'].to_numpy()
    
    InsulinTimeAfter = []
    for i in InsulinTime:
        arr = []
        arr.append(int(str(i).split(':',3)[0]))
        arr.append(int(str(i).split(':',3)[1]))
        arr.append(int(str(i).split(':',3)[2]))
        InsulinTimeAfter.append(arr)
    CGMTimeAfter = []
    for i in CGMTime:
        arr = []
        arr.append(int(str(i).split(':',3)[0]))
        arr.append(int(str(i).split(':',3)[1]))
        arr.append(int(str(i).split(':',3)[2]))
        CGMTimeAfter.append(arr)


    InsulinDateAfter = []
    for i in InsulinDate:
        arr = []
        var = pd.to_datetime(i)
        q= str(var).split()
        arr.append(int(q[0].split('-',3)[0]))
        arr.append(int(q[0].split('-',3)[1]))
        arr.append(int(q[0].split('-',3)[2]))
        InsulinDateAfter.append(arr)


    CGMDateAfter = []
    for i in CGMDate:
        arr = []
        var = pd.to_datetime(i)
        q= str(var).split()
        arr.append(int(q[0].split('-',3)[0]))
        arr.append(int(q[0].split('-',3)[1]))
        arr.append(int(q[0].split('-',3)[2]))
        CGMDateAfter.append(arr)
    
    
    DateandTime = []
    for i in range(0, len(InsulinDateAfter),1):
        arr = []
        arr.append(InsulinDateAfter[i])
        arr.append(InsulinTimeAfter[i])
        DateandTime.append(arr)


    CGMDateandTime = []
    for i in range(0, len(CGMDateAfter),1):
        arr = []
        arr.append(CGMDateAfter[i])
        arr.append(CGMTimeAfter[i])
        CGMDateandTime.append(arr)
    
  
    
    DateAndFood = []
    for i in range(0, len(DateandTime),1):
        arr = []
        arr.append(DateandTime[i])
        arr.append(Eats[i])
        DateAndFood.append(arr)

    DateAndFood = DateAndFood[::-1]


    CGMDateAndFood = []
    for i in range(0, len(CGMDateandTime),1):
        arr = []
        arr.append(CGMDateandTime[i])
        arr.append(eatingBehavior[i])
        arr.append(ISIG[i])
        CGMDateAndFood.append(arr)

    CGMDateAndFood = CGMDateAndFood[::-1]
    
    DataClean = []
    for i in DateAndFood:
        arr = []
        empty = math.isnan(i[1])
        if i[1] > 0 and empty == False:
            arr.append(i[0])
            arr.append(i[1])  
            DataClean.append(arr)
            
            
    print("DataClean")
            
    return DataClean, CGMDateAndFood

In [3]:
#data has been processed

In [4]:
def separateDate(Data,CGM,meal,noMeal,memoize,position):
    
    diff_in_hours = 0
    index = 0
        
    lowerYear = Data[0][0]
    lowerMonth = Data[0][1]
    lowerDay = Data[0][2]
    
        
    lowerHour = Data[1][0]
    lowerMinute = Data[1][1]
    lowerSecond = Data[1][2]
    
    date_1 = str(lowerDay) + '/'+ str(lowerMonth) +'/'+ str(lowerYear) + ' ' +str(lowerHour)+':'+ str(lowerMinute)+':'+ str(lowerSecond)
    date_format_str = '%d/%m/%Y %H:%M:%S'
    start = datetime.strptime(date_1, date_format_str)
    
    
    empty = False 
    considerStretch = False
    memoizationValue = False
    
    for i in range(memoize, len(CGM), 1):
        
        ########################################################################3
        positionYear = position[0][0]
        positionMonth = position[0][1]
        positionDay = position[0][2]


        positionHour = position[1][0]
        positionMinute = position[1][1]
        positionSecond = position[1][2]

        date_3 = str(positionDay) + '/'+ str(positionMonth) +'/'+ str(positionYear) + ' ' +str(positionHour)+':'+ str(positionMinute)+':'+ str(positionSecond)
        memoizationDate = datetime.strptime(date_3, date_format_str)
        
        
        
        
        ##########################################################################
        
        upperYear = CGM[i][0][0][0]
        upperMonth = CGM[i][0][0][1]
        upperDay = CGM[i][0][0][2]

        upperHour = CGM[i][0][1][0]
        upperMinute = CGM[i][0][1][1]
        upperSecond = CGM[i][0][1][2]

        date_2 = str(upperDay) + '/'+ str(upperMonth) +'/'+ str(upperYear) + ' ' +str(upperHour)+':'+ str(upperMinute)+':'+ str(upperSecond)

        end =   datetime.strptime(date_2, date_format_str)
        # Get interval between two timstamps as timedelta object
        
        
        diff = end - start
        # Get interval between two timstamps in hours
        diff_in_hours = diff.total_seconds() / 3600

        #############################################################################
        if diff_in_hours >= 0 and diff_in_hours < 2:
            temp = math.isnan(CGM[i][1])
            if temp == True:
                empty = True
        
        
        if diff_in_hours >= 2 and diff_in_hours <= 4 and empty == True:
            noMeal.append(CGM[i])
            
        if diff_in_hours >= 0 and diff_in_hours < 2 and empty == True:
            meal.append(CGM[i])
            
        if diff_in_hours >= 0 and diff_in_hours < 2 and empty == False:
            considerStretch = True
            
        if diff_in_hours >= 1.5 and diff_in_hours <= 4 and considerStretch == True:
            meal.append(CGM[i])
            
            
        if end >= memoizationDate:
            index = i - 1
            break
            
    return int(index)

In [187]:
def Trainer(array1, array0):
    
    array0 = np.array(array0)
    array1 = np.array(array1)

    X_train = np.concatenate((array0,array1), axis=0)


    labels_0_tr = np.array([0]*array0.shape[0])
    labels_1_tr = np.array([1]*array1.shape[0])

    #the testing classes are grouped under one vector
    y_train = np.concatenate((labels_0_tr,labels_1_tr), axis=0)
    
    
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=.25)


    pca = PCA(n_components=10)# adjust yourself
    pca.fit(X_train)
    pca.fit(X_test)


    X_t_train = pca.transform(X_train)
    X_t_test = pca.transform(X_test)

    clf = SVC(kernel='rbf')
        
    clf.fit(X_t_train, y_train)
    #print('score', clf.score(X_t_test, y_test))
    #print('pred label', clf.predict(X_t_test))
        
    with open ('model_pickle', 'wb') as filename:
        pickle.dump(clf,filename)
        
 
        
        meanTrainingAccuracy = 0
        meanTestingAccuracy = 0
        
        for i in range(1,10,1):
            meanTrainingAccuracy += clf.score(X_t_train, y_train)
            meanTestingAccuracy += clf.score(X_t_test, y_test)
            #print('pred label', modelRetrieved.predict(X_t_test))

        print(meanTrainingAccuracy/i,i)
        print(meanTestingAccuracy/i)


        
        

In [188]:
DataClean, CGMDateAndFood= ReadXlsx('CGMData670GPatient3.xlsx','InsulinAndMealIntake670GPatient3.xlsx')


DataClean


In [189]:
def removeEmpty(meal, noMeal):
    
    array = []
    for i in meal:
        temp = math.isnan(i[1])
        temp1 = math.isnan(i[2])

        if temp == False and temp1 == False:
            
            arr = []
            arr.append(i[0][0][1])
            arr.append(i[0][0][2])
            arr.append(i[0][1][0])
            arr.append(i[0][1][1])
            arr.append(i[1])
            arr.append(i[2])
            array.append(arr)


    array1 = []
    for i in noMeal:
        temp = math.isnan(i[1])
        temp1 = math.isnan(i[2])

        if temp == False and temp1 == False:
            arr = []
            arr.append(i[0][0][1])
            arr.append(i[0][0][2])
            arr.append(i[0][1][0])
            arr.append(i[0][1][1])
            arr.append(i[1])
            arr.append(i[2])
            array1.append(arr)
            
    return array, array1

In [190]:
print(DataClean[0])

[[[2017, 9, 5], [13, 14, 52]], 38.0]


In [191]:
meal = []
noMeal = []
memoize = 0

for i in range(0, len(DataClean), 1):
    
    if i < len(DataClean) - 1:
        memoize = separateDate(DataClean[i][0],CGMDateAndFood,meal,noMeal, memoize, DataClean[i+1][0])

In [192]:
array1, array0 = removeEmpty(meal, noMeal)
from numpy import diff
print(array1[0])

[9, 5, 14, 46, 160.0, 34.55]


In [194]:
glucoseMeal = []
isigMeal = []
for i in array1:
    isigMeal.append(i[3])
    glucoseMeal.append(i[2])

glucoseDeltaMeal = np.arange(len(glucoseMeal))
isigDeltaMeal = np.arange(len(isigMeal))

glucoseGradMeal = np.gradient(glucoseMeal, glucoseDeltaMeal)
isigGradMeal = np.gradient(isigMeal, isigDeltaMeal)
gradMealDifference = abs(glucoseGradMeal - isigGradMeal)

glucose_isigDiffMeal = abs(np.array(glucoseMeal) - np.array(isigMeal))

Eat = []
for i in range(0,len(array1),1):
    arr = []
    arr.append(array1[i][0])
    arr.append(array1[i][1])
    arr.append(array1[i][2])
    arr.append(array1[i][3])
    arr.append(array1[i][4])
    arr.append(array1[i][5])
    arr.append(glucoseGradMeal[i])
    arr.append(isigGradMeal[i])
    arr.append(gradMealDifference[i])
    arr.append(glucose_isigDiffMeal[i])
    Eat.append(arr)


    
    

    
glucoseNoMeal = []
isigNoMeal = []
for i in array0:
    isigNoMeal.append(i[3])
    glucoseNoMeal.append(i[2])

glucoseDeltaNoMeal = np.arange(len(glucoseNoMeal))
isigDeltaNoMeal = np.arange(len(isigNoMeal))

glucoseGradNoMeal = np.gradient(glucoseNoMeal, glucoseDeltaNoMeal)
isigGradNoMeal = np.gradient(isigNoMeal, isigDeltaNoMeal)

gradNoMealDifference = abs(glucoseGradNoMeal - isigGradNoMeal)

glucose_isigDiffNoMeal = abs(np.array(glucoseNoMeal) - np.array(isigNoMeal))


NoEat = []
for i in range(0,len(array0),1):
    arr = []
    arr.append(array1[i][0])
    arr.append(array1[i][1])
    arr.append(array1[i][2])
    arr.append(array1[i][3])
    arr.append(array1[i][4])
    arr.append(array1[i][5])
    arr.append(glucoseGradNoMeal[i])
    arr.append(isigGradNoMeal[i])
    arr.append(gradNoMealDifference[i])
    arr.append(glucose_isigDiffNoMeal[i])
    NoEat.append(arr)

In [195]:
print(Eat[0])

[9, 5, 14, 46, 160.0, 34.55, 0.0, 5.0, 5.0, 32]


In [196]:
Trainer(Eat, NoEat)

0.9500332225913619 9
0.9493822239936228


In [186]:
#kernel='rbf'  
# 0.9427242524916942 9
# 0.9334396173774413

#kernel='sigmoid'
# 0.9227906976744185 9
# 0.9067357512953367

#kernel='poly', degree=8